In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# Import data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Read training data from csv file.
train_data_path = '/kaggle/input/digit-recognizer/train.csv'
train_data = pd.read_csv(train_data_path)
train_data.head()

# Data exploration

In [ ]:
train_labels = train_data[['label']]
train_labels.head()
index = dict()

for label in train_labels.label.unique():
    index[label] = train_labels.loc[train_labels['label'] == label].index

In [ ]:
# Inspect the number of data in each class
count_of_each_label = train_labels.label.value_counts(sort=False)
print(count_of_each_label)
count_of_each_label.plot(kind="bar", rot=0);

In [ ]:
# create train and val set, 80-20 ratio for each class
from sklearn.model_selection import train_test_split

train_index = []
val_index = []
for label in train_labels.label.unique():
    train, val = train_test_split(index[label], train_size=0.8, shuffle=True, random_state=42)
    train_index = [*train_index, *train]
    val_index = [*val_index, *val]
    
train_X = train_data.iloc[train_index].drop(columns=['label'])
train_y = train_data.iloc[train_index][['label']]
val_X = train_data.iloc[val_index].drop(columns=['label'])
val_y = train_data.iloc[val_index][['label']]

In [ ]:
train_X.head()

In [ ]:
train_y.head()

# Create layers

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import datetime

def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    hidden_layer1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)(inputs)
    dropout1 = tf.keras.layers.Dropout(0.2)(hidden_layer1)
    hidden_layer2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.2)(hidden_layer2)
    output = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(dropout2)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=['accuracy'],
    )
    return model

checkpoint1 = ModelCheckpoint("{}.h5".format(datetime.datetime.now()), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint1]

input_shape = ((train_X).shape[1],)
model = create_model(input_shape)
model.summary()

# Training

In [ ]:
history = model.fit(train_X, train_y, validation_data= (val_X, val_y), batch_size=64, epochs=50, callbacks=callbacks_list)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Evaluation

In [ ]:
# Read in the test features.
test_data_path = '/kaggle/input/digit-recognizer/test.csv'
test_data = pd.read_csv(test_data_path)
test_data.head()

In [ ]:
# Predict the all the test features.
test_probs = model.predict(test_data)
test_labels = test_probs.argmax(axis=-1)

In [ ]:
# Generate Submission File.
submission_df = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submission_df.Label = test_labels
submission_df.to_csv("output.csv", index=False)

https://www.kaggle.com/c/digit-recognizer